In [1]:
from ai import figures as fgs

In [2]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [3]:
from plotly.offline import init_notebook_mode
init_notebook_mode(1)

In [5]:
df = pd.read_csv('data/users v3.csv')
df.head()

,signup_date,rate,category,international_win,international_lose,international_draw,turkish_win,turkish_lose,turkish_draw
0,2020-01-27,1664,Class B,50,44,1,58,60,1
1,2020-01-27,1440,Class C,46,60,1,47,43,1
2,2020-01-27,1504,Class C,46,60,1,51,43,1
3,2020-01-29,1680,Class B,63,56,1,45,47,1
4,2020-01-30,1488,Class C,50,58,1,47,46,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   signup_date         100 non-null    object
 1   rate                100 non-null    int64 
 2   category            100 non-null    object
 3   international_win   100 non-null    int64 
 4   international_lose  100 non-null    int64 
 5   international_draw  100 non-null    int64 
 6   turkish_win         100 non-null    int64 
 7   turkish_lose        100 non-null    int64 
 8   turkish_draw        100 non-null    int64 
dtypes: int64(7), object(2)
memory usage: 7.2+ KB


In [10]:
go.FigureWidget(fgs.registration_intensity_histogram(df))

FigureWidget({
    'data': [{'marker': {'color': 'rgba(31, 119, 180, 0.8)', 'line': {'color': 'rgb(0,0,0)', 'w…

In [11]:
go.FigureWidget(fgs.categories_bars(df))

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '19bfb11b-3f4f-44a2-b5af-25136b1852bc',
     …

In [15]:
go.FigureWidget(fgs.categories_ratio_pie(df))

FigureWidget({
    'data': [{'domain': {'x': [0, 0.7]},
              'hole': 0.3,
              'hoverinfo': …

In [16]:
go.FigureWidget(fgs.players_category_histogram(df))

FigureWidget({
    'data': [{'marker': {'line': {'color': 'rgb(0,0,0)', 'width': 1.5}},
              'name': …

In [12]:
go.FigureWidget(fgs.player_iwin_vs_rate_scatter(df))

FigureWidget({
    'data': [{'marker': {'color': 'rgba(31, 119, 180, 0.8)', 'line': {'color': 'rgb(0,0,0)', 'w…

In [13]:
go.FigureWidget(fgs.player_win_ratio_vs_rate(df))

FigureWidget({
    'data': [{'marker': {'color': 'rgba(31, 119, 180, 0.8)', 'line': {'color': 'rgb(0,0,0)', 'w…